In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install tensorflow_text

In [ ]:
#Importing dependencies

import pandas as pd
import numpy as np
import os
import re
from datetime import datetime

from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score as acc
from sklearn.pipeline import make_pipeline as pipe
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer as LemmaT

from xgboost.sklearn import XGBClassifier as XGB
from sklearn.ensemble import RandomForestClassifier as RF, AdaBoostClassifier as ADA, VotingClassifier as Vote, StackingClassifier as Stack
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from transformers import pipeline

import tensorflow as tf
from tensorflow.keras import Sequential,layers
%matplotlib inline
import fasttext
import matplotlib.pyplot as plt

import tensorflow_hub as hub
import tensorflow_text as text
from transformers import DistilBertTokenizer, TFDistilBertModel, TFDistilBertForSequenceClassification, BertTokenizer, TFBertModel, TFBertForSequenceClassification, AutoTokenizer, TFAutoModel, TFAutoModelForSequenceClassification

In [ ]:
max_features=150
max_len=150

In [ ]:
#read training data

train_data=pd.read_csv("../input/intermediate-nlp-mlbuzz/processed_data.csv")

train_data.head()

In [ ]:
# all the industry can be directly predicted if sub categories is known
ind_cat={}
cat_ind={}
for i in range(len(train_data)):
    ind=train_data["Industry"].iloc[i]
    cat=train_data["Sub-category"].iloc[i]
    if ind in ind_cat:
        if cat not in ind_cat[ind]:
            ind_cat[ind].append(cat)
    else:
        ind_cat[ind]=[cat]
        
    if cat not in cat_ind:
        cat_ind[cat]=ind
        
print(ind_cat,cat_ind)

In [ ]:
x=train_data["Article"]
y=train_data["Sub-category"]

enc1hot = OneHotEncoder(handle_unknown='ignore')
enc_ord = OrdinalEncoder()

y1hot=enc1hot.fit_transform(y.to_numpy().reshape(-1,1)).toarray()
y_ord=enc_ord.fit_transform(y.to_numpy().reshape(-1,1))

In [ ]:
# cnt_vec=CountVectorizer(ngram_range=(1,3),max_features=max_len)
# x_vec=cnt_vec.fit_transform(x)

In [ ]:
# # type(x_vec)
# x_vec.shape

In [ ]:
# pca=PCA().fit(x_vec.toarray())
# plt.plot(pca2.explained_variance_, linewidth=2)

In [ ]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# xtokens = tokenizer.batch_encode_plus(list(x),add_special_tokens=True,truncation=True,padding="max_length",max_length=max_len,return_attention_mask=False,return_tensors='np')
# xtokens['input_ids'].shape

In [ ]:
# x_enc=xtokens['input_ids']
# # x_enc=x_enc.reshape((-1,x.shape[1],1))
# x_enc.shape

In [ ]:
# tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
# # bert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')
# bert_model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=13)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')
# bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=13)

# tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
# # bert_model = TFAutoModel.from_pretrained('bert-base-uncased')
# bert_model = TFAutoModelForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=13)



In [ ]:
#tokenization

xtokens = tokenizer.batch_encode_plus(list(x),add_special_tokens=True,truncation=True,padding="max_length",max_length=max_len,return_attention_mask=True,return_tensors='pt')
xtokens

In [ ]:
BATCH_SIZE = 64  # we will use batches of 32

dataset = tf.data.Dataset.from_tensor_slices((xtokens.input_ids, xtokens.attention_mask, y1hot))

def map_func(input_ids, masks, labels):
    return {'input_ids': input_ids, 'attention_mask': masks}, labels

# using map method to apply map_func to dataset
dataset = dataset.map(map_func)

# shuffle data and batch it
dataset = dataset.shuffle(100000).batch(BATCH_SIZE)

# get the length of the batched dataset
DS_LEN = len([0 for batch in dataset])
SPLIT = 0.9  # 90-10 split

train = dataset.take(round(DS_LEN*SPLIT))  # get first 90% of batches
val = dataset.skip(round(DS_LEN*SPLIT))  # skip first 90% and keep final 10%

del dataset  # optionally, delete dataset to free up disk-space

In [ ]:
# # #test train split

# xtrain,xtest,ytrain,ytest=train_test_split(x,y1hot,test_size=0.3,random_state=7)
# print(len(xtrain),len(xtest),len(ytrain),len(ytest))

In [ ]:
# ##Model

# def model():
#     input_ids = tf.keras.layers.Input(shape=(), name='input_ids', dtype='int32')
#     mask = tf.keras.layers.Input(shape=(), name='attention_mask', dtype='int32')
# #     x=tokenizer.encode_plus(x,add_special_tokens=True,truncation=True,padding="max_length",max_length=512,return_attention_mask=True,return_tensors='tf')
#     x=bert_model(input_ids, attention_mask=mask)
#     x=tf.keras.layers.BatchNormalization()(x.pooler_output)
#     x=tf.keras.layers.Dropout(0.25)(x)
#     x=tf.keras.layers.Dense(13, activation='softmax')(x)

    

# deepNN=model()

In [ ]:
#model

input_ids = tf.keras.layers.Input(shape=(max_len,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(max_len,), name='attention_mask', dtype='int32')

# we consume the last_hidden_state tensor from bert (discarding pooled_outputs)
X = bert_model(input_ids, attention_mask=mask).last_hidden_state
# X = tf.keras.layers.Flatten()(X)
X = tf.keras.layers.LSTM(64)(X)

X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.Dense(64, activation='relu')(X)
X = tf.keras.layers.Dropout(0.2)(X)

X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.Dense(32, activation='relu')(X)
X = tf.keras.layers.Dropout(0.2)(X)

y = tf.keras.layers.Dense(13, activation='softmax', name='outputs')(X)

# define input and output layers of our model
model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

# freeze the BERT layer - otherwise we will be training 100M+ parameters...
model.layers[2].trainable = False

model.summary()

optimizer = tf.keras.optimizers.Adam(0.01)
loss = tf.keras.losses.CategoricalCrossentropy()  # categorical = one-hot
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[acc])
early_stop=tf.keras.callbacks.EarlyStopping(monitor="val_acc",min_delta=0.01,patience=5,restore_best_weights=True)
checkpoint=tf.keras.callbacks.ModelCheckpoint(filepath="./chckpoints/",save_weights_only=False,save_best_only=True,verbose=1)

checkpoint_dir='../input/mlbuzz-intermediate-model-checkpoint/checkpoints/checkpoints/variables/variables'
# latest = tf.train.latest_checkpoint(checkpoint_dir)
# print(latest)
model.load_weights(checkpoint_dir)


history = model.fit(train,validation_data=val,batch_size=64,callbacks=[early_stop,checkpoint],epochs=100,shuffle=True)

In [ ]:
# optimizer = tf.keras.optimizers.Adam(0.01)
# loss = tf.keras.losses.CategoricalCrossentropy()  # categorical = one-hot
# acc = tf.keras.metrics.CategoricalAccuracy('accuracy')


# bert_model.compile(optimizer=optimizer, loss=loss, metrics=[acc])
# callback=tf.keras.callbacks.EarlyStopping(monitor="val_acc",min_delta=0.01,patience=5,restore_best_weights=True)
# history = bert_model.fit(train,validation_data=val,batch_size=64,callbacks=[callback],epochs=20,shuffle=True)

In [ ]:
# # lstm


# def LSTM_model():
#     model=Sequential()
# #     model.add(layers.Embedding(max_len,256))
# #     model.add(LSTM(256,activation='tanh',recurrent_activation='sigmoid',dropout=0.25,recurrent_dropout=0.25,return_sequences=True,input_shape=(None,1)))
# #     model.add(layers.Bidirectional(layers.LSTM(64,activation='tanh',recurrent_activation='sigmoid',dropout=0.25,recurrent_dropout=0.25)))
    
#     model.add(layers.LSTM(128,activation='tanh',recurrent_activation='sigmoid',dropout=0.25,recurrent_dropout=0.25))
    
#     model.add(layers.Dense(128,activation='relu'))
#     model.add(layers.Dropout(0.25))
#     model.add(layers.BatchNormalization())
    
# #     model.add(layers.Flatten())
    
#     model.add(layers.Dense(64,activation='relu'))
#     model.add(layers.Dropout(0.25))
#     model.add(layers.BatchNormalization())
    
#     model.add(layers.Dense(13,activation='softmax'))
    
#     model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    
#     return model

In [ ]:
# model=LSTM_model()
# model.summary()

In [ ]:
# callback=tf.keras.callbacks.EarlyStopping(monitor="val_accuracy",min_delta=0.01,patience=30,restore_best_weights=True)
# history = model.fit(x_enc,y1hot,batch_size=32,callbacks=[callback],epochs=300,shuffle=True,validation_split=0.2)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train','Validation'])
plt.show()

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train','Validation'])
plt.show()